In [ ]:
import random
random.choices([0, 1], weights=[3/5, 2/5])[0]
#если возник вопрос как сгенерировать 0 (3/5) или 1 (2/5) с разной вероятностью, то на питоне можно сделать так

In [ ]:
#Если есть разный масштаб между признаками, то необходимо делать стандартизацию!!!

In [1]:
#Евклидово, Манхэттеновское, max-метрика
import pandas as pd
import numpy as np
from numpy import NaN

# Создадим датафрейм с пропущенным значением признака Р для объекта А
df = pd.DataFrame({'P1':[3,5,4,5], 'P2':[4,5,3,4], 'P3':[5,5,3,3], 'P4':[3,4,2,3], 'P':[4,3,5,NaN]}, index=['A1', 'A2', 'A3', 'A'])

# Посчитаем метрики
dict_metrics = {'A1':[], 'A2':[], 'A3':[]}
for i in df.index[:-1]:
    dict_metrics[i].append(np.power((df.loc['A'][:-1]-df.loc[i][:-1]).pow(2).sum(), 0.5).round(2)) # считаем Евклидово расстояние
    dict_metrics[i].append((df.loc['A'][:-1]-df.loc[i][:-1]).abs().sum()) # считаем Манхэттеновское расстояние
    dict_metrics[i].append((df.loc['A'][:-1]-df.loc[i][:-1]).abs().max()) # считаем max-метрику

metrics = pd.DataFrame(dict_metrics, index=['Euclid', 'Manhatten', 'Max'])

# Считаем варианты значений для каждой метрики
dict_value = {'Euclid':[], 'Manhatten':[], 'Max':[]}
for i in metrics.index:
    norm_mul = (1/((1/metrics.loc[i]).sum())) # нормирующий множитель
    similarity = ((df.loc[:]['P'][:-1]/metrics.loc[i]).sum()) # значение признака * мера близости(=величина, обратно пропорциональная мере расстояния)
    value_P = (norm_mul*similarity).round(2)
    dict_value[i].append(value_P)
    print(f'значение признака P для А по метрике {i}: {value_P}')

значение признака P для А по метрике Euclid: 4.13
значение признака P для А по метрике Manhatten: 4.1
значение признака P для А по метрике Max: 4.25


In [2]:
import pandas as pd
import numpy as np

df = pd.DataFrame(
    data = {
        "Объекты": ["A1", "A2", "A3", "A"],
        "P1": [3, 5, 4, 5],
        "P2": [4, 5, 3, 4],
        "P3": [5, 5, 3, 3],
        "P4": [3, 4, 2, 3],
        "P":  [4, 3, 5, None]
    }
)
df.set_index("Объекты", inplace=True)
display(df)


# Euclidean metric
metrics = pd.DataFrame(data={"euclidean": np.sqrt(np.sum((df.loc['A'] - df.loc['A1':'A3'])**2, axis=1))})
# manhattan metric
metrics['manhattan'] = np.sum(np.abs(df.loc['A'] - df.loc['A1':'A3']), axis=1)
# max
metrics['max'] = np.max(
    np.abs(df.loc['A'] - df.loc['A1': 'A3']),
    axis=1
)
display('метрики', metrics)

# нормирующий множитель
coeff = 1 / np.sum(1 / metrics, axis=0)
display('нормирующий множитель', coeff)


# меры близости для метрик
p1 = 1 / metrics    # расчет мер близости
display('меры близости', p1) 
weights = p1.T @ df.loc['A1':'A3', 'P'] #умножение мер на значения известных призоков P и суммируем
display(weights)
unknown_feature = weights * coeff
display(unknown_feature)

,P1,P2,P3,P4,P
Объекты,,,,,
A1,3,4,5,3,4.0
A2,5,5,5,4,3.0
A3,4,3,3,2,5.0
A,5,4,3,3,NaN


'метрики'

,euclidean,manhattan,max
Объекты,,,
A1,2.828427,4.0,2.0
A2,2.449490,4.0,2.0
A3,1.732051,3.0,1.0


'нормирующий множитель'

euclidean    0.746741
manhattan    1.200000
max          0.500000
dtype: float64

'меры близости'

,euclidean,manhattan,max
Объекты,,,
A1,0.353553,0.250000,0.5
A2,0.408248,0.250000,0.5
A3,0.577350,0.333333,1.0


euclidean    5.525710
manhattan    3.416667
max          8.500000
dtype: float64

euclidean    4.126275
manhattan    4.100000
max          4.250000
dtype: float64

In [20]:
from neulab.recover import replace_missing_with_distance

#C:\Users\skapr\AppData\Roaming\Python\Python39\site-packages\neulab

d = {'P1': [3, 5, 4, 5], 'P2': [4, 5, 3, 4], 'P3': [5, 5, 3, 3], 'P4': [3, 4, 2, 3], 'P5': [4, 3, 5, np.NaN]}
df = pd.DataFrame(data = d)

# Euclid
#euclid_m = MetricRestore(df, row_start=0, row_end=9, metric='euclid')
euclid_m, d_e = replace_missing_with_distance(df, metric = 'euclidean')
# Manhattan
mnht_m, d_mn = replace_missing_with_distance(df, metric = 'manhattan')
# Max
mx_m, d_mx = replace_missing_with_distance(df, metric = 'max')

print(d_e, d_mn, d_mx)

3.84 3.83 3.82


In [38]:
def replace_missing_with_corrcoef(*vectors):
    # Combine the input vectors into a single 2D array
    if len(vectors) > 1:
        vector = np.vstack(vectors)
    else:
        vector = np.array(vectors)[0]

    missing_values = np.isnan(vector)
    num_missing_values = np.count_nonzero(missing_values)
    if num_missing_values == len(vector):
        raise ValueError("All values in vector are missing.")
        
    # Assuming that missing values are represented by NaNs
    missing_mask = np.isnan(vector)
    missing_indices = np.where(missing_mask)[0]
    nonmissing_indices = np.where(~missing_mask)[0]
    
    if len(missing_indices) == 0:
        # If there are no non-missing values, return the input vector and NaN correlation
        return vector, np.nan
    
    # Calculate the correlation coefficient between non-missing values and missing indices
    corr_coef = np.corrcoef(nonmissing_indices, vector[nonmissing_indices])
    
    if np.isnan(corr_coef):
        # If the correlation coefficient is NaN, return the input vector and NaN correlation
        return vector, np.nan
    
    # Calculate the new values for missing indices using the correlation coefficient
    new_values = (missing_indices - nonmissing_indices.mean()) * corr_coef + vector[nonmissing_indices].mean()
    
    # Replace the missing values with the new values
    new_vector = vector.copy()
    new_vector[missing_mask] = new_values
    
    return new_vector, round(corr_coef, 6)

In [22]:
from sklearn.preprocessing import StandardScaler

def recover_by_metric(data, metric):
    data_scaled = StandardScaler().fit_transform(data.values)
    clear_data = data_scaled[~np.isnan(data_scaled).any(axis=1)]
    clear_data_origin = data[~data.isna().any(axis=1)]
    nan_data = data_scaled[np.isnan(data_scaled).any(axis=1)]
    nan_data_origin = data[data.isna().any(axis=1)]
    recovered_data = np.zeros(nan_data.shape)
    for i, nan_row in enumerate(nan_data):
        clear_indices = np.where(~np.isnan(nan_row))[0]
        metrics = np.array([metric(obj[clear_indices], nan_row[clear_indices]) for obj in clear_data])
        args = np.argsort(metrics)
        if metrics[args[0]] == 0:
            recovered_data[i, :] = clear_data_origin.iloc[args[:np.sum(metrics == 0)], :].values.mean(axis=0)
        else:
            recovered_data[i, :] = ((1 / metrics) @ clear_data_origin.values) / np.sum(1 / metrics)
            recovered_data[i, clear_indices] = nan_data_origin.iloc[i, clear_indices]

    return pd.DataFrame(np.concatenate([clear_data_origin.values, recovered_data], axis=0), 
                        columns=data.columns, index=data.index)

In [ ]:
#В качестве metric функцию, измеряющую какую-нибудь метрику. Например, из библиотеки sklearn можно подгрузить функции

from sklearn.metrics import mean_squared_error

recover_by_metric(dataFrame, mean_squared_error)

#Либо свою руками функцию можете написать
def my_MSE(y_true, y_predicted):
    return ((y_true - y_predicted) ** 2).mean()

recover_by_metric(dataFrame, my_MSE)

In [40]:
import pandas as pd
import numpy as np
from numpy import NaN
from sklearn import preprocessing

df = pd.DataFrame({'bust':[99,89,91,91,86,97,NaN], 
              'waidt':[56,58,64,51,56,53,51], 
              'hips':[91,89,91,91,84,86,91], 
              'height':[160,157,165,170,157,175,165], 
              'weight':[58,48,54,54,44,56,54]}, index=['W1','W2','W3','W4','W5','W6','W7'])

def pred_kk_by_feature(df, pred_object, pred_feature):
    temp = df.corr()[pred_feature]
    KK = temp[temp.index != pred_feature]
    KK # КК для всех признаков

    temp = df[df.index != pred_object].mean(axis=0)
    mean_all_feature = temp[temp.index != pred_feature]
    mean_all_feature # Среднее значение для всех признаков (кроме предсказываемого) по всем наблюдениям (кроме предсказываемого)

    all_feature_pred_object = df.loc[pred_object][df.loc[pred_object].index != pred_feature] # Значения известных признаков предсказываемого наблюдения
    dif_mult_kk = ((all_feature_pred_object-mean_all_feature)*KK).sum() # сумма произведений разности известных значений и средних, и КК
    mean_feature_pred_object = df[pred_feature].mean() # среднее значение признака
    pred_feature_object = mean_feature_pred_object+1/KK.abs().sum()*dif_mult_kk # получаем предсказание пропущенного значения

    print(f'Значение признака, рассчитанное с помощью КК (по признакам) : {round(pred_feature_object, 2)}')
    return(pred_feature_object)

def pred_kk_by_object(df, pred_object, pred_feature):
    temp = df.transpose().corr()[pred_object] # транспонируем данные для рассчета КК между объектами
    KK = temp[temp.index != pred_object]
    
    temp = df.transpose()[df.transpose().index != pred_feature].mean(axis=0) # среднее значение всех признаков (кроме предсказываемого) в известных объектах
    mean_all_object = temp[temp.index != pred_object]
    
    all_object_pred_feature = df.transpose().loc[pred_feature][df.transpose().loc[pred_feature].index != pred_object] # значения предсказываемого признака для известных объектов
    dif_mult_kk = ((all_object_pred_feature-mean_all_object)*KK).sum() # сумма произведений разности известных значений и средних, и КК
    norm_mult = 1/KK.abs().sum() # нормирующий множитель
    mean_W = df.mean(axis=1)[pred_object] # среднее значение всех признаков для W7
    
    pred_feature_object = mean_W+norm_mult*dif_mult_kk # получаем предсказание пропущенного значения
    
    print(f'Значение признака, рассчитанное с помощью КК (по объектам) : {round(pred_feature_object, 2)}')
    return(pred_feature_object)

def pred_metric(metric_name, df, pred_object, pred_feature):
    min_max_scaler = preprocessing.MinMaxScaler() # создаем объект нормализации
    # затем вызываем метод fit_transform, передаем в него датафрейм с нужными колонками (преобразованный в массив), 
    # полученный массив преобразуем обратно в датафрейм и присваиваем его значению изначальных колок
    df[df.columns[df.columns != pred_feature]] = pd.DataFrame(min_max_scaler.fit_transform(df.transpose()[df.transpose().index != pred_feature].transpose().values), columns = df.columns[df.columns != pred_feature], index = df.index)

    temp = df.transpose()[pred_object] 
    feature_pred_object = temp[temp.index != pred_feature] # признаки для предсказываемого объекта

    dict_metric = {} # создаем словарь метрик, преобразуем его в датафрейм
    for name_object in df.index[df.index != pred_object]:
        t = df.loc[name_object]
        if metric_name == 'euclid':
            dict_metric[name_object] = np.power((feature_pred_object - t[t.index != pred_feature]).pow(2).sum(), 0.5).round(2)
        elif metric_name == 'manhatten':
            dict_metric[name_object] = (feature_pred_object - t[t.index != pred_feature]).abs().sum()
        else:
            dict_metric[name_object] = (feature_pred_object - t[t.index != pred_feature]).abs().max()
    metric = pd.DataFrame(data = dict_metric, index = [metric_name]).transpose()

    temp = df.transpose()[df.transpose().index == pred_feature].transpose() 
    feature_other_objects = temp[temp.index != pred_object] # предсказываемый признак для других объектов

    norm_mul = 1/((1/metric).sum()).item() # нормирующий множитель
    similarity = (feature_other_objects.values/metric.values).sum() # значение признака * мера близости(=величина, обратно пропорциональная мере расстояния)
    pred_feature_object = norm_mul*similarity
    print(f'Значение признака, рассчитанное по метрике {metric_name}: {round(pred_feature_object, 2)}')
    return pred_feature_object

pred_object = 'W7'
pred_feature = 'bust'

methods = {'pred_kk_by_feature': pred_kk_by_feature(df, pred_object, pred_feature),
           'pred_kk_by_object': pred_kk_by_object(df, pred_object, pred_feature), 
           'pred_metric_euclid': pred_metric('euclid', df, pred_object, pred_feature), 
           'pred_metric_manhatten': pred_metric('manhatten', df, pred_object, pred_feature), 
           'pred_metric_max_metric': pred_metric('max_metric', df, pred_object, pred_feature)} 

for method in methods.values():
    method

Значение признака, рассчитанное с помощью КК (по признакам) : 94.21
Значение признака, рассчитанное с помощью КК (по объектам) : 92.09
Значение признака, рассчитанное по метрике euclid: 92.61
Значение признака, рассчитанное по метрике manhatten: 92.25
Значение признака, рассчитанное по метрике max_metric: 92.79


In [41]:
from scipy.spatial import distance

a = (0, 1, 2)
b = (2, 1, 0)
dst = distance.euclidean(a, b)
dst

2.8284271247461903

In [42]:
from scipy.spatial.distance import cityblock
import pandas as pd

a = (0, 1, 2)
b = (2, 1, 0)
dst = cityblock(a, b)
dst

4

In [ ]:
import numpy as np
from scipy.spatial.distance import pdist, squareform
A = np.random.randint(-5, 5, [500, 3])
D = squareform(pdist(A))
N = np.max(D)
I = np.argmax(D)
I_row, I_col = np.unravel_index(I, D.shape)

In [43]:
p = [1,0,5,2,2]
for i in p:
    i = (i - min(p)) / (max(p) - min(p))
    print(i, end=', ')

0.2, 0.0, 1.0, 0.4, 0.4, 